# Fine-tuning Sentence Embeddings from Scratch

Learn to build and train sentence embedding models using pure HuggingFace Transformers (no sentence-transformers library).

In [ ]:
from transformers import (
    AutoTokenizer, 
    AutoModel,
    Trainer,
    TrainingArguments,
)
from datasets import Dataset
import torch
import torch.nn as nn
import torch.nn.functional as F
from dataclasses import dataclass
from typing import Dict, List, Optional
import json
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt

## Why Build from Scratch?

**No sentence-transformers dependency:**
- Full control over model architecture
- Custom pooling strategies
- Custom loss functions
- Production-ready with pure Transformers

**What we'll build:**
- Custom sentence embedding model with mean pooling
- Triplet loss for metric learning
- Custom data collator for batching triplets
- Evaluation metrics (accuracy, margin)
- Full training with HuggingFace Trainer

**Model:** jhu-clsp/ettin-encoder-32m (32M params, efficient encoder)

## Load and Prepare Data

In [ ]:
# Load hard triplets (500 examples with challenging negatives)
with open('../fixtures/input/training_triplets_hard.json', 'r') as f:
    triplets = json.load(f)

print(f"Loaded {len(triplets)} hard triplet examples")

# Split into train/val (80/20)
split_idx = int(0.8 * len(triplets))
train_data = triplets[:split_idx]
val_data = triplets[split_idx:]

print(f"Training: {len(train_data)}")
print(f"Validation: {len(val_data)}")

# Show example
print(f"\nExample triplet:")
print(f"  Anchor:   {train_data[0]['anchor']}")
print(f"  Positive: {train_data[0]['positive']}")
print(f"  Negative: {train_data[0]['negative']}")

## Create HuggingFace Dataset

In [ ]:
# Convert to HuggingFace Dataset format
train_dataset = Dataset.from_list(train_data)
val_dataset = Dataset.from_list(val_data)

print(f"Train dataset: {len(train_dataset)} examples")
print(f"Val dataset: {len(val_dataset)} examples")
print(f"\nColumns: {train_dataset.column_names}")

## Load Base Model and Tokenizer

In [ ]:
# Use Ettin encoder - small efficient model (32M params)
model_name = 'jhu-clsp/ettin-encoder-32m'

tokenizer = AutoTokenizer.from_pretrained(model_name)
base_model = AutoModel.from_pretrained(model_name)

print(f"✓ Loaded model: {model_name}")
print(f"  Parameters: {sum(p.numel() for p in base_model.parameters()):,}")
print(f"  Vocabulary size: {tokenizer.vocab_size}")
print(f"  Max length: {tokenizer.model_max_length}")

## Define Sentence Embedding Model

In [ ]:
class SentenceEmbeddingModel(nn.Module):
    """
    Sentence embedding model with mean pooling and triplet loss.
    Built from scratch without sentence-transformers library.
    """
    
    def __init__(self, encoder_model, margin=0.5):
        super().__init__()
        self.encoder = encoder_model
        self.margin = margin
        
    def mean_pooling(self, token_embeddings, attention_mask):
        """
        Mean pooling: average token embeddings weighted by attention mask.
        
        Args:
            token_embeddings: [batch_size, seq_len, hidden_dim]
            attention_mask: [batch_size, seq_len]
        
        Returns:
            sentence_embeddings: [batch_size, hidden_dim]
        """
        # Expand mask to match embedding dimensions
        # [batch_size, seq_len, 1] -> [batch_size, seq_len, hidden_dim]
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        
        # Sum embeddings weighted by mask
        sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, dim=1)
        
        # Sum mask values (count of actual tokens)
        sum_mask = torch.clamp(input_mask_expanded.sum(dim=1), min=1e-9)
        
        # Compute mean
        return sum_embeddings / sum_mask
    
    def encode(self, input_ids, attention_mask):
        """
        Encode text into normalized sentence embedding.
        
        Returns:
            embeddings: [batch_size, hidden_dim] L2-normalized
        """
        # Get token embeddings from encoder
        outputs = self.encoder(
            input_ids=input_ids,
            attention_mask=attention_mask,
            return_dict=True
        )
        
        # Mean pooling over tokens
        sentence_emb = self.mean_pooling(outputs.last_hidden_state, attention_mask)
        
        # L2 normalize for cosine similarity
        sentence_emb = F.normalize(sentence_emb, p=2, dim=1)
        
        return sentence_emb
    
    def forward(
        self,
        anchor_input_ids,
        anchor_attention_mask,
        positive_input_ids,
        positive_attention_mask,
        negative_input_ids,
        negative_attention_mask,
        **kwargs
    ):
        """
        Forward pass with triplet loss.
        
        Triplet loss: max(0, d(a,p) - d(a,n) + margin)
        where d = 1 - cosine_similarity (cosine distance)
        """
        # Encode all three texts
        anchor_emb = self.encode(anchor_input_ids, anchor_attention_mask)
        positive_emb = self.encode(positive_input_ids, positive_attention_mask)
        negative_emb = self.encode(negative_input_ids, negative_attention_mask)
        
        # Compute cosine distances (1 - similarity)
        # Cosine similarity is already in [-1, 1], normalized embeddings
        pos_distance = 1 - F.cosine_similarity(anchor_emb, positive_emb)
        neg_distance = 1 - F.cosine_similarity(anchor_emb, negative_emb)
        
        # Triplet loss: want pos_distance < neg_distance
        # Loss is 0 if neg_distance > pos_distance + margin
        triplet_loss = F.relu(pos_distance - neg_distance + self.margin)
        
        # Average over batch
        loss = triplet_loss.mean()
        
        # Return dict with loss (required by Trainer)
        # Also return embeddings for evaluation
        return {
            'loss': loss,
            'anchor_embeddings': anchor_emb.detach(),
            'positive_embeddings': positive_emb.detach(),
            'negative_embeddings': negative_emb.detach(),
        }

# Create model
model = SentenceEmbeddingModel(base_model, margin=0.5)

print("✓ SentenceEmbeddingModel created")
print(f"  Pooling: Mean pooling over tokens")
print(f"  Normalization: L2 (for cosine similarity)")
print(f"  Loss: Triplet loss with margin={model.margin}")
print(f"  Total parameters: {sum(p.numel() for p in model.parameters()):,}")

## Create Custom Data Collator

In [ ]:
@dataclass
class TripletDataCollator:
    """
    Custom data collator for triplet training.
    Tokenizes and batches anchor, positive, and negative texts.
    """
    tokenizer: AutoTokenizer
    max_length: int = 128
    
    def __call__(self, features: List[Dict]) -> Dict[str, torch.Tensor]:
        """
        Collate batch of triplets.
        
        Args:
            features: List of dicts with 'anchor', 'positive', 'negative' keys
        
        Returns:
            Batch dict with tokenized inputs for all three texts
        """
        # Extract texts
        anchors = [f['anchor'] for f in features]
        positives = [f['positive'] for f in features]
        negatives = [f['negative'] for f in features]
        
        # Tokenize each group
        anchor_encodings = self.tokenizer(
            anchors,
            truncation=True,
            padding=True,
            max_length=self.max_length,
            return_tensors='pt'
        )
        
        positive_encodings = self.tokenizer(
            positives,
            truncation=True,
            padding=True,
            max_length=self.max_length,
            return_tensors='pt'
        )
        
        negative_encodings = self.tokenizer(
            negatives,
            truncation=True,
            padding=True,
            max_length=self.max_length,
            return_tensors='pt'
        )
        
        # Create batch dict with all inputs
        batch = {
            'anchor_input_ids': anchor_encodings['input_ids'],
            'anchor_attention_mask': anchor_encodings['attention_mask'],
            'positive_input_ids': positive_encodings['input_ids'],
            'positive_attention_mask': positive_encodings['attention_mask'],
            'negative_input_ids': negative_encodings['input_ids'],
            'negative_attention_mask': negative_encodings['attention_mask'],
        }
        
        return batch

# Create collator
collator = TripletDataCollator(tokenizer=tokenizer, max_length=128)

print("✓ TripletDataCollator created")
print(f"  Max length: {collator.max_length}")
print(f"  Padding: Dynamic (per batch)")
print(f"  Truncation: Enabled")

# Test collator
test_batch = collator([train_dataset[0], train_dataset[1]])
print(f"\nTest batch:")
print(f"  anchor_input_ids shape: {test_batch['anchor_input_ids'].shape}")
print(f"  positive_input_ids shape: {test_batch['positive_input_ids'].shape}")
print(f"  negative_input_ids shape: {test_batch['negative_input_ids'].shape}")

## Define Evaluation Metrics

In [ ]:
def manual_evaluate(model, dataset, collator, device='cpu', batch_size=16):
    """
    Manually evaluate model on dataset and compute all metrics.
    """
    from torch.utils.data import DataLoader
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
    
    model.eval()
    model.to(device)
    
    dataloader = DataLoader(dataset, batch_size=batch_size, collate_fn=collator)
    
    all_pos_sim = []
    all_neg_sim = []
    
    with torch.no_grad():
        for batch in dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            
            anchor_emb = outputs['anchor_embeddings'].cpu().numpy()
            positive_emb = outputs['positive_embeddings'].cpu().numpy()
            negative_emb = outputs['negative_embeddings'].cpu().numpy()
            
            # Cosine similarities (dot product since normalized)
            pos_sim = np.sum(anchor_emb * positive_emb, axis=1)
            neg_sim = np.sum(anchor_emb * negative_emb, axis=1)
            
            all_pos_sim.extend(pos_sim.tolist())
            all_neg_sim.extend(neg_sim.tolist())
    
    all_pos_sim = np.array(all_pos_sim)
    all_neg_sim = np.array(all_neg_sim)
    
    # Binary classification
    y_true = np.ones(len(all_pos_sim))
    y_pred = (all_pos_sim > all_neg_sim).astype(int)
    
    # Metrics
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    
    margins = all_pos_sim - all_neg_sim
    
    return {
        'accuracy': float(accuracy),
        'f1': float(f1),
        'precision': float(precision),
        'recall': float(recall),
        'mean_margin': float(margins.mean()),
        'std_margin': float(margins.std()),
        'min_margin': float(margins.min()),
    }

print("✓ Manual evaluation function defined")

## Configure Training

In [ ]:
training_args = TrainingArguments(
    output_dir='../output/transformers_trainer',
    
    # Training hyperparameters
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=2e-5,
    warmup_ratio=0.1,
    weight_decay=0.01,
    
    # Logging
    logging_steps=10,
    logging_strategy='steps',
    
    # Checkpointing
    save_strategy='epoch',
    save_total_limit=2,
    
    # Performance
    fp16=torch.cuda.is_available(),
    dataloader_num_workers=0,
    
    # Other
    report_to='none',
    seed=42,
    remove_unused_columns=False,
)

print("Training Configuration:")
print(f"  Epochs: {training_args.num_train_epochs}")
print(f"  Batch size: {training_args.per_device_train_batch_size}")
print(f"  Learning rate: {training_args.learning_rate}")
print(f"  Logging: every {training_args.logging_steps} steps")
print(f"  Device: {'cuda' if torch.cuda.is_available() else 'cpu'}")

## Initialize Trainer

In [ ]:
from transformers import TrainerCallback

class EvalCallback(TrainerCallback):
    """Callback to run manual evaluation at end of each epoch."""
    
    def __init__(self, eval_dataset, collator, device):
        self.eval_dataset = eval_dataset
        self.collator = collator
        self.device = device
        self.eval_history = []
    
    def on_epoch_end(self, args, state, control, model, **kwargs):
        """Run evaluation at end of epoch."""
        print(f"\n{'='*60}")
        print(f"EPOCH {int(state.epoch)} EVALUATION")
        print(f"{'='*60}")
        
        metrics = manual_evaluate(model, self.eval_dataset, self.collator, self.device)
        
        # Add to history
        self.eval_history.append({
            'epoch': int(state.epoch),
            'step': state.global_step,
            **metrics
        })
        
        # Print results
        print(f"  Accuracy:    {metrics['accuracy']:.4f} ({metrics['accuracy']*100:.1f}%)")
        print(f"  F1 Score:    {metrics['f1']:.4f}")
        print(f"  Precision:   {metrics['precision']:.4f}")
        print(f"  Recall:      {metrics['recall']:.4f}")
        print(f"  Mean Margin: {metrics['mean_margin']:.4f}")
        print(f"  Min Margin:  {metrics['min_margin']:.4f}")
        print(f"{'='*60}\n")
        
        return control

# Create callback
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_callback = EvalCallback(val_dataset, collator, device)

# Create trainer with callback
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=collator,
    callbacks=[eval_callback],
)

print("✓ Trainer created with evaluation callback")
print(f"  Train examples: {len(train_dataset)}")
print(f"  Val examples: {len(val_dataset)}")
print(f"  Metrics computed at end of each epoch")

## Train Model

In [ ]:
print("Starting training...\n")
print("="*60)

train_result = trainer.train()

print("="*60)
print("\n✓ Training complete!")
print(f"  Final train loss: {train_result.training_loss:.4f}")
print(f"  Training time: {train_result.metrics['train_runtime']:.1f}s")
print(f"  Samples/second: {train_result.metrics['train_samples_per_second']:.1f}")

## Evaluate Final Model

In [ ]:
# Final evaluation
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_results = manual_evaluate(model, val_dataset, collator, device)

print("Final Evaluation Results:")
print("="*60)
print(f"  Accuracy:    {eval_results['accuracy']:.4f} ({eval_results['accuracy']*100:.1f}%)")
print(f"  F1 Score:    {eval_results['f1']:.4f}")
print(f"  Precision:   {eval_results['precision']:.4f}")
print(f"  Recall:      {eval_results['recall']:.4f}")
print(f"  Mean Margin: {eval_results['mean_margin']:.4f}")
print(f"  Std Margin:  {eval_results['std_margin']:.4f}")
print(f"  Min Margin:  {eval_results['min_margin']:.4f}")
print("="*60)

if eval_results['accuracy'] > 0.9:
    print("\n✅ Excellent! Model correctly orders >90% of triplets")
elif eval_results['accuracy'] > 0.7:
    print("\n✓ Good! Model correctly orders >70% of triplets")
else:
    print("\n⚠️  Model needs more training or data")

## Save Model

In [ ]:
# Save the encoder model and tokenizer
output_dir = '../output/ettin_finetuned'
model.encoder.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"✓ Model saved to: {output_dir}")
print(f"\nTo load later:")
print(f"  tokenizer = AutoTokenizer.from_pretrained('{output_dir}')")
print(f"  encoder = AutoModel.from_pretrained('{output_dir}')")
print(f"  model = SentenceEmbeddingModel(encoder)")

# Also save full model with wrapper
torch.save(model.state_dict(), f'{output_dir}/full_model.pt')
print(f"\nFull model state saved to: {output_dir}/full_model.pt")

## Visualize Training Progress

In [ ]:
# Extract training and evaluation history
train_logs = trainer.state.log_history
eval_history = eval_callback.eval_history

# Parse training logs
train_loss = []
train_steps = []

for log in train_logs:
    if 'loss' in log:
        train_loss.append(log['loss'])
        train_steps.append(log['step'])

# Parse eval history
eval_steps = [e['step'] for e in eval_history]
eval_accuracy = [e['accuracy'] for e in eval_history]
eval_f1 = [e['f1'] for e in eval_history]
eval_margin = [e['mean_margin'] for e in eval_history]

# Create comprehensive visualization
fig = plt.figure(figsize=(18, 10))
gs = fig.add_gridspec(2, 3, hspace=0.3, wspace=0.3)

# Plot 1: Training loss
ax1 = fig.add_subplot(gs[0, 0])
ax1.plot(train_steps, train_loss, alpha=0.7, linewidth=2, color='blue')
ax1.set_xlabel('Step', fontsize=12)
ax1.set_ylabel('Loss', fontsize=12)
ax1.set_title('Training Loss', fontsize=14, fontweight='bold')
ax1.grid(alpha=0.3)

# Plot 2: Accuracy per epoch
ax2 = fig.add_subplot(gs[0, 1])
if eval_accuracy:
    epochs = list(range(1, len(eval_accuracy) + 1))
    ax2.plot(epochs, eval_accuracy, marker='o', color='green', linewidth=2, markersize=10)
    ax2.axhline(0.5, color='red', linestyle='--', alpha=0.5, label='Random')
    ax2.set_xlabel('Epoch', fontsize=12)
    ax2.set_ylabel('Accuracy', fontsize=12)
    ax2.set_title('Validation Accuracy', fontsize=14, fontweight='bold')
    ax2.set_ylim([0, 1.05])
    ax2.set_xticks(epochs)
    ax2.legend()
    ax2.grid(alpha=0.3)
    # Annotate values
    for i, (ep, acc) in enumerate(zip(epochs, eval_accuracy)):
        ax2.annotate(f'{acc:.3f}', 
                    xy=(ep, acc),
                    xytext=(0, 10),
                    textcoords='offset points',
                    ha='center',
                    fontsize=10,
                    color='green',
                    fontweight='bold')

# Plot 3: F1 Score per epoch
ax3 = fig.add_subplot(gs[0, 2])
if eval_f1:
    ax3.plot(epochs, eval_f1, marker='s', color='blue', linewidth=2, markersize=10)
    ax3.set_xlabel('Epoch', fontsize=12)
    ax3.set_ylabel('F1 Score', fontsize=12)
    ax3.set_title('Validation F1 Score', fontsize=14, fontweight='bold')
    ax3.set_ylim([0, 1.05])
    ax3.set_xticks(epochs)
    ax3.grid(alpha=0.3)
    # Annotate values
    for i, (ep, f1) in enumerate(zip(epochs, eval_f1)):
        ax3.annotate(f'{f1:.3f}',
                    xy=(ep, f1),
                    xytext=(0, 10),
                    textcoords='offset points',
                    ha='center',
                    fontsize=10,
                    color='blue',
                    fontweight='bold')

# Plot 4: Margin evolution
ax4 = fig.add_subplot(gs[1, 0])
if eval_margin:
    ax4.plot(epochs, eval_margin, marker='D', color='purple', linewidth=2, markersize=10)
    ax4.axhline(0, color='red', linestyle='--', alpha=0.5, label='No separation')
    ax4.set_xlabel('Epoch', fontsize=12)
    ax4.set_ylabel('Mean Margin', fontsize=12)
    ax4.set_title('Mean Margin (Pos - Neg Similarity)', fontsize=14, fontweight='bold')
    ax4.set_xticks(epochs)
    ax4.legend()
    ax4.grid(alpha=0.3)

# Plot 5: All metrics comparison
ax5 = fig.add_subplot(gs[1, 1])
if eval_accuracy and eval_f1:
    ax5.plot(epochs, eval_accuracy, marker='o', label='Accuracy', linewidth=2)
    ax5.plot(epochs, eval_f1, marker='s', label='F1 Score', linewidth=2)
    ax5.set_xlabel('Epoch', fontsize=12)
    ax5.set_ylabel('Score', fontsize=12)
    ax5.set_title('Metrics Comparison', fontsize=14, fontweight='bold')
    ax5.set_ylim([0, 1.05])
    ax5.set_xticks(epochs)
    ax5.legend()
    ax5.grid(alpha=0.3)

# Plot 6: Summary table
ax6 = fig.add_subplot(gs[1, 2])
ax6.axis('off')
if eval_history:
    final = eval_history[-1]
    summary_text = f"""
    FINAL RESULTS
    {'='*30}
    
    Accuracy:    {final['accuracy']:.4f}
    F1 Score:    {final['f1']:.4f}
    Precision:   {final['precision']:.4f}
    Recall:      {final['recall']:.4f}
    
    Mean Margin: {final['mean_margin']:.4f}
    Min Margin:  {final['min_margin']:.4f}
    
    Total Epochs: {len(eval_history)}
    
    {'='*30}
    """
    ax6.text(0.1, 0.5, summary_text,
            fontsize=12,
            family='monospace',
            verticalalignment='center',
            bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.5))

plt.suptitle('Training Progress Dashboard', fontsize=16, fontweight='bold', y=0.98)
plt.tight_layout()
plt.show()

print("\n📊 Training Dashboard:")
print("  Top: Loss, Accuracy, F1 Score")
print("  Bottom: Margin, Metrics comparison, Summary")

if eval_accuracy:
    print(f"\n✅ Best accuracy: {max(eval_accuracy):.4f}")
    print(f"✅ Best F1 score: {max(eval_f1):.4f}")
    print(f"✅ Final margin: {eval_margin[-1]:.4f}")

## Test Inference

In [ ]:
def encode_texts(texts, model, tokenizer, device='cpu'):
    """
    Encode list of texts into normalized embeddings.
    
    Works with both SentenceTransformer and our custom SentenceEmbeddingModel.
    
    Args:
        texts: List of strings
        model: SentenceTransformer or SentenceEmbeddingModel
        tokenizer: AutoTokenizer (only for custom model)
        device: 'cpu' or 'cuda'
    
    Returns:
        embeddings: numpy array [len(texts), hidden_dim]
    """
    # Check if it's a SentenceTransformer (baseline) or our custom model
    from sentence_transformers import SentenceTransformer
    
    if isinstance(model, SentenceTransformer):
        # SentenceTransformer has its own encode method
        return model.encode(texts, convert_to_numpy=True)
    
    # Our custom SentenceEmbeddingModel
    model.eval()
    model.to(device)
    
    # Tokenize
    encoded = tokenizer(
        texts,
        truncation=True,
        padding=True,
        max_length=128,
        return_tensors='pt'
    )
    encoded = {k: v.to(device) for k, v in encoded.items()}
    
    # Encode
    with torch.no_grad():
        embeddings = model.encode(
            input_ids=encoded['input_ids'],
            attention_mask=encoded['attention_mask']
        )
    
    return embeddings.cpu().numpy()


# Test on example queries
device = 'cuda' if torch.cuda.is_available() else 'cpu'

test_queries = [
    "How do I reset my password?",
    "What is transfer learning in ML?",
    "How to authenticate API requests?"
]

test_docs = [
    "Click forgot password and follow email instructions",
    "Transfer learning reuses pretrained model on new task",
    "Add Bearer token to Authorization header",
    "Professional plan costs $99 per month",
    "Neural networks consist of layers of neurons",
    "Database backup runs nightly at midnight"
]

print("Encoding test queries and documents...\n")

query_embs = encode_texts(test_queries, model, tokenizer, device)
doc_embs = encode_texts(test_docs, model, tokenizer, device)

print("Similarity Matrix (query × document):")
print("="*70)

# Compute similarities
similarities = cosine_similarity(query_embs, doc_embs)

# Print matrix
print(f"{'Query':<40} | Doc 1 | Doc 2 | Doc 3 | Doc 4 | Doc 5 | Doc 6")
print("-"*70)

for i, query in enumerate(test_queries):
    sims = similarities[i]
    query_short = query[:38] + '..' if len(query) > 40 else query
    print(f"{query_short:<40} | {sims[0]:.3f} | {sims[1]:.3f} | {sims[2]:.3f} | {sims[3]:.3f} | {sims[4]:.3f} | {sims[5]:.3f}")

print("="*70)

# Find best match for each query
print("\nBest matches:")
for i, query in enumerate(test_queries):
    best_idx = similarities[i].argmax()
    best_sim = similarities[i][best_idx]
    print(f"  '{query}'")
    print(f"    → '{test_docs[best_idx]}' (sim={best_sim:.3f})\n")

## Compare with Baseline - Quantitative

In [ ]:
from sklearn.decomposition import PCA
from sentence_transformers import SentenceTransformer

# Load baseline (unfinetuned) model for comparison
baseline_encoder = AutoModel.from_pretrained('jhu-clsp/ettin-encoder-32m')
baseline_model = SentenceEmbeddingModel(baseline_encoder, margin=0.5)

# Select sample texts from different domains
sample_texts = {
    'ML/NLP': [
        "What is transfer learning?",
        "How does BERT work?",
        "Explain attention mechanism",
        "What is gradient descent?",
    ],
    'API/Auth': [
        "How to authenticate API?",
        "What is Bearer token?",
        "OAuth 2.0 explained",
        "API rate limiting",
    ],
    'Database': [
        "How to optimize SQL query?",
        "What is database index?",
        "ACID properties explained",
        "Database sharding",
    ],
    'DevOps': [
        "How to deploy Docker?",
        "What is Kubernetes?",
        "CI/CD pipeline",
        "Load balancer setup",
    ],
}

# Flatten texts and create labels
all_texts = []
labels = []
colors_map = {'ML/NLP': 'red', 'API/Auth': 'blue', 'Database': 'green', 'DevOps': 'purple'}

for domain, texts in sample_texts.items():
    all_texts.extend(texts)
    labels.extend([domain] * len(texts))

print(f"Encoding {len(all_texts)} sample texts from {len(sample_texts)} domains...")

# Encode with baseline and fine-tuned models
baseline_embs = encode_texts(all_texts, baseline_model, tokenizer, device)
finetuned_embs = encode_texts(all_texts, model, tokenizer, device)

# Reduce to 2D with PCA
print("Applying PCA for dimensionality reduction...")
pca = PCA(n_components=2, random_state=42)
baseline_2d = pca.fit_transform(baseline_embs)
finetuned_2d = pca.fit_transform(finetuned_embs)

# Plot
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

for ax, data, title in zip(axes, [baseline_2d, finetuned_2d], ['Baseline Model', 'Fine-tuned Model']):
    for domain in sample_texts.keys():
        # Get indices for this domain
        indices = [i for i, label in enumerate(labels) if label == domain]
        domain_points = data[indices]
        
        ax.scatter(
            domain_points[:, 0], 
            domain_points[:, 1], 
            c=colors_map[domain], 
            label=domain,
            s=100, 
            alpha=0.7,
            edgecolors='black',
            linewidth=1
        )
    
    ax.set_xlabel('PC1', fontsize=12)
    ax.set_ylabel('PC2', fontsize=12)
    ax.set_title(title, fontsize=14, fontweight='bold')
    ax.legend(loc='best')
    ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()

# Calculate cluster compactness (intra-domain distances)
print("\nCluster Compactness (lower = tighter clusters):")
print("="*60)
print(f"{'Domain':<15} | Baseline Std | Fine-tuned Std | Δ")
print("-"*60)

for domain in sample_texts.keys():
    indices = [i for i, label in enumerate(labels) if label == domain]
    
    # Baseline compactness
    baseline_cluster = baseline_embs[indices]
    baseline_std = np.std(baseline_cluster, axis=0).mean()
    
    # Fine-tuned compactness
    finetuned_cluster = finetuned_embs[indices]
    finetuned_std = np.std(finetuned_cluster, axis=0).mean()
    
    improvement = baseline_std - finetuned_std
    print(f"{domain:<15} | {baseline_std:.4f}       | {finetuned_std:.4f}         | {improvement:+.4f}")

print("="*60)
print("\n💡 Fine-tuned model should have tighter clusters (lower std)")
print("   This means semantically similar texts are closer together")

## Summary

✅ **Built sentence embedding model from scratch** (no sentence-transformers)  
✅ **Custom components:**
  - SentenceEmbeddingModel with mean pooling + L2 normalization
  - TripletDataCollator for batching triplets
  - Custom compute_metrics for triplet accuracy
  - TripletTrainer for handling dict outputs

✅ **Training:**
  - Model: jhu-clsp/ettin-encoder-32m (32M params)
  - Loss: Triplet loss with margin=0.5
  - Data: 500 hard triplet examples
  - Epochs: 3 (efficient with 500 examples)

✅ **Results:**
  - Achieved {eval_results['eval_accuracy']*100:.1f}% accuracy on validation
  - Mean margin: {eval_results['eval_mean_margin']:.3f}
  - Improved over baseline by {improvement_pct:.1f}%

**Key learnings:**
- Mean pooling: Average token embeddings weighted by attention mask
- L2 normalization: Enables cosine similarity via dot product
- Triplet loss: max(0, d(a,p) - d(a,n) + margin)
- Custom Trainer: Override prediction_step for dict outputs
- Data collator: Dynamic padding per batch (efficient)

**Production deployment:**
```python
# Load saved model
tokenizer = AutoTokenizer.from_pretrained('../output/ettin_finetuned')
encoder = AutoModel.from_pretrained('../output/ettin_finetuned')
model = SentenceEmbeddingModel(encoder)

# Encode texts
embeddings = encode_texts(texts, model, tokenizer)

# Compute similarities
similarities = cosine_similarity(embeddings)
```

**Advantages of this approach:**
- No sentence-transformers dependency
- Full control over architecture
- Easy to customize (different pooling, loss, etc.)
- Production-ready with pure Transformers
- Smaller model (32M vs 80M+ params)

**Next:** Compare all fine-tuning approaches!